# Create a retreat mask from basins and Calfin

In [1]:
import geopandas as gp
gp.options.io_engine = "pyogrio"
import pandas as pd
from tqdm.auto import tqdm
import xarray as xr
from geocube.api.core import make_geocube
import numpy as np
import rioxarray as rxr
from joblib import Parallel, delayed
from pathlib import Path
from typing import Union, Dict

## Set horizontal grid resolution

In [2]:
resolution = 450

## Read IMBIE and Calfin using geopandas

In [3]:
imbie = gp.read_file("../data/imbie/GRE_Basins_IMBIE2_v1.3_w_shelves.gpkg").to_crs("EPSG:3413")
calfin = gp.read_file("../data/calfin/termini_1972-2019_Greenland_closed_v1.0.shp")

In [4]:
geom_valid = calfin.geometry.make_valid()
calfin.geometry = geom_valid
calfin_dissolved = calfin.dissolve()

In [21]:
imbie_dissolved = imbie.dissolve()

In [22]:
date= pd.DatetimeIndex(calfin["Date"])

In [23]:
calfin_ds = calfin.set_index(date).groupby(by=pd.Grouper(freq='ME'))
nt = len(calfin_ds)

In [24]:
imbie_union = imbie_dissolved.union(calfin_dissolved)

In [25]:
x_min = -653000
x_max = 879700
y_min = -632750
y_max = -3384350
bbox = [x_min, y_min, x_max, y_max]
geom = {
    "type": "Polygon",
    "crs": {"properties": {"name": "EPSG:3413"}},
    "bbox": bbox,
    "coordinates": [[
        (x_min, y_min), 
        (x_max, y_min), 
        (x_max, y_max), 
        (x_min, y_max),
        (x_min, y_min)  # Close the loop by repeating the first point
    ]]
}

In [26]:
def create_ds(ds1, ds2, date, geom: Dict, resolution: float = 450, crs: str = "EPSG:3413"):
    if len(ds1) > 0:
        diff = ds2.difference(ds1.dissolve().buffer(5))
        n = len(diff)
        diff_df = {"land_ice_area_fraction_retreat": np.ones(n)}
        diff_gp = gp.GeoDataFrame(data=diff_df, geometry=diff, crs=crs)
        ds = make_geocube(vector_data=diff_gp, geom=geom, resolution=(resolution, resolution))
        ds.land_ice_area_fraction_retreat["units"] = "1"
        ds = ds.expand_dims(time=[date])
        fn = Path(f"frontretreat_g{resolution}m_{date.year}-{date.month}.nc", encoding={"zlib": True, "complevel": 2})
        ds.to_netcdf(fn)
        del ds
        return fn

In [30]:
n_jobs = 8
result = Parallel(n_jobs=n_jobs)(
    delayed(create_ds)(ds,
        imbie_union, date, geom, resolution=resolution)
    for date, ds in tqdm(calfin_ds))

  0%|          | 0/562 [00:00<?, ?it/s]

## Merge files and add time bounds

This could probably be done with *xarray* but setting the time axis and time_bounds correctly appears hard.

In [31]:
fn = Path(f"pism_g{resolution}m_frontretreat_calfin_1972_2019.nc")
!rm -rf $fn
!cdo -O -f nc4 -z zip_2 settbounds,1month -setmisstoc,0 -mergetime frontretreat_g$gridm_*.nc $fn
!rm frontretreat_g$gridm_*

cdo(1) setmisstoc: Process started
cdo(2) mergetime: Process started
cdo    settbounds: Processed 6187640085 values from 1 variable over 297 timesteps [84.26s 961MB]


In [29]:
ls

GIS_mass_accounting.pdf
GRE_Basins_IMBIE2_v1.3.cpg
GRE_Basins_IMBIE2_v1.3.dbf
GRE_Basins_IMBIE2_v1.3.prj
GRE_Basins_IMBIE2_v1.3.qml
GRE_Basins_IMBIE2_v1.3.qpj
GRE_Basins_IMBIE2_v1.3.shp
GRE_Basins_IMBIE2_v1.3.shx
GRE_G0240_1985_2018_IDW_EXP_1.nc
GRE_G1800_1985_2018_IDW_EXP_1.nc
GRE_G900_1985_2018_IDW_EXP_1.nc
Ocean_Forcing.ipynb
Untitled.ipynb
analysis.ipynb
create_retreat_mask.ipynb
foo.nc
idw_merge_its_live.ipynb
jak_idw.nc
pism_g450m_frontretreat_calfin_1972_2019.nc
pism_g450m_frontretreat_calfin_1972_2019.nc.aux.xml
